In [1]:
import pandas as pd
import numpy as np

In [4]:
# import xslx file
url = 'tfl-daily-cycle-hires.xlsx'
df_bike = pd.read_excel(url, sheet_name='Data')
df_bike.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
df_bike.head()

,Day,Number of Bicycle Hires
0,2015-01-01,9615
1,2015-01-02,15389
2,2015-01-03,5779
3,2015-01-04,9367
4,2015-01-05,20566


In [6]:
df_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Day                      1827 non-null   datetime64[ns]
 1   Number of Bicycle Hires  1827 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 28.7 KB


In [7]:
#rename the columns
df_bike.columns = [col.lower() for col in df_bike.columns]
df_bike.columns = [col.replace(" ", "_") for col in df_bike.columns]
df_bike.rename(columns= {
    'day': 'date'
}, inplace=True)

In [9]:
# get day, month and year from date
df_bike['day'] = df_bike['date'].dt.day
df_bike['month'] = df_bike['date'].dt.month
df_bike['year'] = df_bike['date'].dt.year

In [20]:
# group by month and day
grouped_df_bike = df_bike.groupby(['month', 'day']).agg(
    number_of_bicycle_hires_mean=('number_of_bicycle_hires', 'mean'),
    number_of_bicycle_hires_std=('number_of_bicycle_hires', 'std')
).reset_index()


In [21]:
grouped_df_bike

,month,day,number_of_bicycle_hires_mean,number_of_bicycle_hires_std
0,1,1,10267.666667,2462.755584
1,1,2,13745.800000,4598.386043
2,1,3,14422.400000,8329.192356
3,1,4,18107.800000,5168.602877
4,1,5,20549.200000,3887.664427
...,...,...,...,...
361,12,27,9992.800000,2759.433764
362,12,28,12565.000000,1250.650231
363,12,29,12725.000000,2769.551047
364,12,30,12811.800000,2021.540551


In [ ]:
#index of 
#grouped_df_bike = grouped_df_bike.set_index(['month', 'day'])

In [15]:
# group by year, month and day
grouped_year_df_bike = df_bike.groupby(['year', 'month', 'day']).agg(
    number_of_bicycle_hires_sum=('number_of_bicycle_hires', 'sum')
).reset_index()



In [28]:
# drop feb 29
condition1 = grouped_year_df_bike['month'] == 2
condition2 = grouped_year_df_bike['day'] == 29

index_feb29 = grouped_year_df_bike[condition1 & condition2].index

grouped_year_df_bike.drop(index_feb29, inplace=True)

grouped_df_bike.reset_index(drop=True, inplace=True)
grouped_year_df_bike.reset_index(drop=True, inplace=True)

In [23]:
grouped_df_bike

,month,day,bicycle_hires_ave,bicycle_hires_std
0,1,1,10268,2462.755584
1,1,2,13746,4598.386043
2,1,3,14422,8329.192356
3,1,4,18108,5168.602877
4,1,5,20549,3887.664427
...,...,...,...,...
361,12,27,9993,2759.433764
362,12,28,12565,1250.650231
363,12,29,12725,2769.551047
364,12,30,12812,2021.540551


In [30]:
grouped_df_bike.dropna(subset='bicycle_hires_std', inplace=True)

In [ ]:
# data cleaning and finetuning 
grouped_year_df_bike.rename(columns=({'number_of_bicycle_hires_sum': 'bicycle_hires'}), inplace=True)
grouped_df_bike.rename(columns=({'number_of_bicycle_hires_mean': 'bicycle_hires_ave', 'number_of_bicycle_hires_std': 'bicycle_hires_std'}), inplace=True)
grouped_df_bike['bicycle_hires_ave'] = grouped_df_bike['bicycle_hires_ave'].apply(round)
grouped_df_bike['bicycle_hires_std'] = grouped_df_bike['bicycle_hires_std'].apply(round)

In [32]:
# get the index of na row -->
grouped_df_bike.loc[pd.isna(grouped_df_bike["bicycle_hires_std"]), :].index

Index([], dtype='int64')

In [33]:
grouped_year_df_bike.head(10)

,year,month,day,bicycle_hires
0,2015,1,1,9615
1,2015,1,2,15389
2,2015,1,3,5779
3,2015,1,4,9367
4,2015,1,5,20566
5,2015,1,6,20765
6,2015,1,7,22504
7,2015,1,8,15769
8,2015,1,9,22285
9,2015,1,10,14877


In [34]:
grouped_df_bike.head(10)

,month,day,bicycle_hires_ave,bicycle_hires_std
0,1,1,10268,2463
1,1,2,13746,4598
2,1,3,14422,8329
3,1,4,18108,5169
4,1,5,20549,3888
5,1,6,18224,4062
6,1,7,18507,5652
7,1,8,20271,5748
8,1,9,24650,9447
9,1,10,22048,6785


In [35]:
#merge the two dataframes
merged_df_bike = pd.merge(grouped_year_df_bike, grouped_df_bike, on=['month', 'day'], how='left')

In [55]:
#calculate Z-score
"""Compute Z-Scores (Standardized Deviation)
Z-score tells you how many standard deviations away a particular day's value is from the mean:
Z-score = (actual_value - mean)/std

Interpretations -->
z ≈ 0: The day is typical (very close to average).
|z| > 1: The day is somewhat unusual.
|z| > 2: The day is statistically significant — quite rare (could indicate an event, weather impact, etc.).
z < 0: Fewer hires than usual.
z > 0: More hires than usual.
"""

merged_df_bike["z_score"] = (merged_df_bike["bicycle_hires"] - merged_df_bike["bicycle_hires_ave"]
) / merged_df_bike["bicycle_hires_std"]

# Z-score interpretations

def z_score_interpret(z_score):
    if z_score == 0:
        return 'The day is typical'
    elif abs(z_score) > 1 and abs(z_score) < 1.6:
        return 'The day is unusual'
    elif abs(z_score) > 1.6:
        return 'The day is significantly unusual'
    elif z_score < 0:
        return 'Fewer hires than usual'
    elif z_score > 0:
        return 'More hires than usual'
    else:
        return 'Nan'

# call it with function 

merged_df_bike['z_score_interpretation'] = merged_df_bike['z_score'].apply(z_score_interpret)

In [56]:
# calculate deviation from the average in percentage and round it 2 decimal places
merged_df_bike['deviation_from_average_pct'] = round(((merged_df_bike['bicycle_hires'] - merged_df_bike['bicycle_hires_ave']) / merged_df_bike['bicycle_hires_ave'] * 100),2)

In [43]:
merged_df_bike.sample(10)

,year,month,day,bicycle_hires,bicycle_hires_ave,bicycle_hires_std,z_score,z_score_interpretation,deviation_from_average_pct
1181,2018,3,28,15800,21997,9942,-0.623315,Fewer hires than usual,-28.17
1101,2018,1,7,12259,18507,5652,-1.105449,The day is unusual,-33.76
1070,2017,12,7,24810,23657,4699,0.245371,More hires than usual,4.87
637,2016,9,30,34140,30265,4232,0.915643,More hires than usual,12.80
98,2015,4,9,31671,27248,10488,0.421720,More hires than usual,16.23
927,2017,7,17,41811,40869,2176,0.432904,More hires than usual,2.30
989,2017,9,17,25300,32680,6395,-1.154027,The day is unusual,-22.58
1416,2018,11,18,9585,20920,8911,-1.272023,The day is unusual,-54.18
791,2017,3,3,22914,17808,8282,0.616518,More hires than usual,28.67
1719,2019,9,17,39025,32680,6395,0.992181,More hires than usual,19.42


In [51]:
merged_df_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        1826 non-null   int32  
 1   month                       1826 non-null   int32  
 2   day                         1826 non-null   int32  
 3   bicycle_hires               1826 non-null   int64  
 4   bicycle_hires_ave           1826 non-null   int64  
 5   bicycle_hires_std           1826 non-null   int64  
 6   z_score                     1826 non-null   float64
 7   z_score_interpretation      1826 non-null   object 
 8   deviation_from_average_pct  1826 non-null   float64
dtypes: float64(2), int32(3), int64(3), object(1)
memory usage: 107.1+ KB


In [57]:
merged_df_bike['z_score_interpretation'].value_counts()

z_score_interpretation
More hires than usual               822
The day is unusual                  467
Fewer hires than usual              441
The day is significantly unusual     96
Name: count, dtype: int64

In [58]:
# export xlsx file
merged_df_bike.to_excel('final_df_bike_hires.xlsx', index=False)

